<a href="https://colab.research.google.com/github/ritusingh-29/Hybrid-Legal-Document-Summarization/blob/main/hybrid_bert_part2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ==============================================================================
# 1. SETUP: Install all necessary libraries
# ==============================================================================
print("Installing libraries...")
!pip install transformers datasets pandas matplotlib seaborn rouge-score sacrebleu bert-score sentencepiece torch tqdm evaluate --quiet

import os
import datasets
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import (
    BartTokenizer, BartForConditionalGeneration,
    get_linear_schedule_with_warmup
)
from torch.optim import AdamW
from datasets import load_dataset
import evaluate
from tqdm.notebook import tqdm
from google.colab import drive

print("--- All libraries installed and imported successfully! ---")

# ==============================================================================
# 2. CONNECT TO YOUR GOOGLE DRIVE
# ==============================================================================
print("\n[STEP 1] Connecting to Google Drive...")
try:
    drive.mount('/content/drive', force_remount=True)
    print("Google Drive successfully mounted!")
except Exception as e:
    print(f"Error mounting drive: {e}")
    # Stop the script if drive can't be mounted
    raise SystemExit("Google Drive mount failed. Cannot proceed.")

Installing libraries...
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.4 MB/s eta 0:00:00
--- All libraries installed and imported successfully! ---

[STEP 1] Connecting to Google Drive...
Mounted at /content/drive
Google Drive successfully mounted!


In [ ]:
# ==============================================================================
# 3. LOAD YOUR PRE-PROCESSED HYBRID DATA (The "Import" Step)
# ==============================================================================
# This is the path where you saved your condensed data
hybrid_data_path = "/content/drive/MyDrive/Hybrid_Dataset_Saved_BERT"

print(f"\n[STEP 2] Loading your condensed data from: {hybrid_data_path}")
try:
    loaded_hybrid_train_dataset = datasets.load_from_disk(f"{hybrid_data_path}/train")
    loaded_hybrid_test_dataset = datasets.load_from_disk(f"{hybrid_data_path}/test")
    print("--- Condensed data loaded successfully! ---")
    print(loaded_hybrid_train_dataset)
except Exception as e:
    print(f"Error loading data: {e}")
    print("Please check that the path is correct and the data was saved properly.")
    # Stop the script if data can't be loaded
    raise SystemExit("Failed to load hybrid data.")

# ==============================================================================
# 4. DATA PREPARATION CLASS (Same as before)
# ==============================================================================
# This class will now wrap your loaded data and prepare it for the model
class BillSumDataset(Dataset):
    def __init__(self, docs, summaries, tokenizer,
                 max_input_len=1024,  # Use 1024, as the text is already condensed
                 max_output_len=128):
        self.docs = docs
        self.summaries = summaries
        self.tokenizer = tokenizer
        self.max_input_len = max_input_len
        self.max_output_len = max_output_len

    def __len__(self):
        return len(self.docs)

    def __getitem__(self, idx):
        doc = str(self.docs[idx])
        summary = str(self.summaries[idx])

        inputs = self.tokenizer(
            doc,
            max_length=self.max_input_len,
            truncation=True,
            padding="max_length",
            return_tensors="pt"
        )
        targets = self.tokenizer(
            summary,
            max_length=self.max_output_len,
            truncation=True,
            padding="max_length",
            return_tensors="pt"
        )

        labels = targets["input_ids"].squeeze()
        labels[labels == self.tokenizer.pad_token_id] = -100

        return {
            "input_ids": inputs["input_ids"].squeeze(),
            "attention_mask": inputs["attention_mask"].squeeze(),
            "labels": labels
        }

# ==============================================================================
# 5. MODEL LOADING & TRAINING SETUP (Using Your Best Hyperparameters)
# ==============================================================================
print("\n[STEP 3] Loading a fresh BART model and preparing for training...")

# Load a FRESH, pre-trained BART model
model_name = "facebook/bart-base"
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)

# --- Create the PyTorch Datasets ---
# We feed our LOADED hybrid texts into the dataset class
train_dataset = BillSumDataset(loaded_hybrid_train_dataset['text'], loaded_hybrid_train_dataset['summary'], tokenizer)
test_dataset = BillSumDataset(loaded_hybrid_test_dataset['text'], loaded_hybrid_test_dataset['summary'], tokenizer)

# --- Use the BEST Hyperparameters from your tuning (Run 2) ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

batch_size = 8  # <-- Using 8 to be safe and avoid GPU RAM crashes
epochs = 4
lr = 5e-5       # <-- Your best learning rate

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
optimizer = AdamW(model.parameters(), lr=lr)

total_steps = epochs * len(train_loader)
scheduler = get_linear_schedule_with_warmup(
    optimizer, num_warmup_steps=int(0.1 * total_steps), num_training_steps=total_steps
)

# ==============================================================================
# 6. FINE-TUNING LOOP (with Checkpointing)
# ==============================================================================

# --- Mount Drive and define a base save path *before* the loop ---
from google.colab import drive
drive.mount('/content/drive')
# Create one main folder for all your epoch checkpoints
base_save_path = "/content/drive/MyDrive/My_ML_Project/Hybrid_Model_Checkpoints"
os.makedirs(base_save_path, exist_ok=True)


print(f"\n[STEP 4] Starting BERT-HYBRID model training for {epochs} epochs on {device}...")
model.train()
for epoch in range(epochs):
    print(f"--- Epoch {epoch+1}/{epochs} ---")
    epoch_loss = 0.0

    # --- This is your inner training loop (it stays the same) ---
    for batch in tqdm(train_loader, desc=f"Training Hybrid Epoch {epoch+1}"):
        optimizer.zero_grad()
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels,
            return_dict=True
        )
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        scheduler.step()
        epoch_loss += loss.item()

    avg_loss = epoch_loss / len(train_loader)
    print(f"Average training loss for epoch {epoch+1}: {avg_loss:.4f}")

    # --- NEW CHECKPOINTING CODE (runs at the end of each epoch) ---
    print(f"\nCheckpointing model after Epoch {epoch+1}...")

    # Create a unique folder for this epoch's model
    epoch_save_path = f"{base_save_path}/hybrid_model_epoch_{epoch+1}"
    os.makedirs(epoch_save_path, exist_ok=True)

    model.save_pretrained(epoch_save_path)
    tokenizer.save_pretrained(epoch_save_path)

    print(f"Model for Epoch {epoch+1} successfully saved to: {epoch_save_path}\n")
    # --- End of new checkpointing code ---

print("\n--- Full Training Complete! ---")

# ==============================================================================
# 7. EVALUATION
# ==============================================================================
print("\n[STEP 5] Starting Final Evaluation on the BERT-HYBRID model...")
rouge = evaluate.load("rouge")
bleu = evaluate.load("sacrebleu")
bertscore = evaluate.load("bertscore")

model.eval()
preds, refs = [], []
test_loader = DataLoader(test_dataset, batch_size=4) # Using a small batch size for evaluation to be safe

with torch.no_grad():
    for batch in tqdm(test_loader, desc="Evaluating Hybrid Model"):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"]

        gen_ids = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_length=128,
            num_beams=4,
            early_stopping=True
        )

        batch_preds = tokenizer.batch_decode(gen_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True)
        labels[labels == -100] = tokenizer.pad_token_id
        batch_refs = tokenizer.batch_decode(labels, skip_special_tokens=True, clean_up_tokenization_spaces=True)

        preds.extend(batch_preds)
        refs.extend(batch_refs)

# Compute metrics
rouge_scores = rouge.compute(predictions=preds, references=refs)
bleu_score = bleu.compute(predictions=preds, references=[[r] for r in refs])
bert_scores = bertscore.compute(predictions=preds, references=refs, lang="en")

print("\n\n=== FINAL EVALUATION RESULTS (BERT-HYBRID MODEL) ===")
print("ROUGE:", rouge_scores)
print("BLEU:", bleu_score)
print("BERTScore F1 mean:", sum(bert_scores['f1']) / len(bert_scores['f1']))

# ==============================================================================
# 8. SAVE YOUR NEW MODEL (Important!)
# ==============================================================================
print("\n[STEP 6] Saving the fine-tuned BERT-HYBRID model to Google Drive...")
try:
    # --- Save to a NEW folder ---
    output_dir = "/content/drive/MyDrive/My_ML_Project/bert_hybrid_bart_model" #<-- New permanent location
    os.makedirs(output_dir, exist_ok=True)

    model.save_pretrained(output_dir)
    tokenizer.save_pretrained(output_dir)

    print(f"\nModel and tokenizer permanently saved at {output_dir}")
except Exception as e:
    print(f"Error saving to Drive: {e}. Model was not saved permanently.")

print("\n--- Hybrid Model Experiment Complete! ---")


[STEP 2] Loading your condensed data from: /content/drive/MyDrive/Hybrid_Dataset_Saved_BERT
--- Condensed data loaded successfully! ---
Dataset({
    features: ['text', 'summary'],
    num_rows: 18949
})

[STEP 3] Loading a fresh BART model and preparing for training...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

Training Hybrid Epoch 3:   0%|          | 0/2369 [00:00<?, ?it/s]

Average training loss for epoch 3: 1.5666

Checkpointing model after Epoch 3...
Model for Epoch 3 successfully saved to: /content/drive/MyDrive/My_ML_Project/Hybrid_Model_Checkpoints/hybrid_model_epoch_3

--- Epoch 4/4 ---


Training Hybrid Epoch 4:   0%|          | 0/2369 [00:00<?, ?it/s]

Average training loss for epoch 4: 1.4608

Checkpointing model after Epoch 4...
Model for Epoch 4 successfully saved to: /content/drive/MyDrive/My_ML_Project/Hybrid_Model_Checkpoints/hybrid_model_epoch_4


--- Full Training Complete! ---

[STEP 5] Starting Final Evaluation on the BERT-HYBRID model...


Evaluating Hybrid Model:   0%|          | 0/818 [00:00<?, ?it/s]